In [2]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyRegressor
from sklearn.utils import all_estimators
from sklearn.base import RegressorMixin

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [3]:
cols_cible_type = ["PumpSecondsOnSite", "TurnoutTimeSeconds", "TravelTimeSeconds"]
cols_cible = [
    ["PumpSecondsOnSite_min", "PumpSecondsOnSite_mean", "PumpSecondsOnSite_max"],
    ["TurnoutTimeSeconds_min", "TurnoutTimeSeconds_mean", "TurnoutTimeSeconds_max"],
    ["TravelTimeSeconds_min", "TravelTimeSeconds_mean", "TravelTimeSeconds_max"]
    # NumPumpsAttending??
]

def load_df(col_cible_type):
    df = pd.read_csv(f"../data/_df_ready_{col_cible_type}.csv", sep=";", low_memory=False)
    # Contrôle
    # display(df.head(3))
    # display(df.info())
    return df

In [ ]:
# Créez le cercle de corrélation
# def draw_correlation_circle(df_charges_factorielles, pca, arrow_length=0.1, label_rotation=0):
#     fig, ax = plt.subplots(figsize=(8, 8))
#     for i, var in enumerate(df_charges_factorielles.columns):
#         x = df_charges_factorielles.loc[0, var]
#         y = df_charges_factorielles.loc[1, var]
#         ax.arrow(0, 0, x, y, head_width=arrow_length, head_length=arrow_length, fc='gray', ec='gray')
#         ax.text(x*1.15, y*1.15, var, ha='center', va='center', rotation=label_rotation, fontsize=9)
#     circle = plt.Circle((0, 0), 1, facecolor='none', edgecolor='black')
#     ax.add_artist(circle)
#     ax.set_xlim(-1.1, 1.1)
#     ax.set_ylim(-1.1, 1.1)
#     ax.set_aspect('equal', adjustable='box')
#     ax.set_xlabel('Axe 1 (PC1)')
#     ax.set_ylabel('Axe 2 (PC2)')
#     ax.set_title('Cercle des Corrélations')
#     plt.grid()
#     plt.show()

ignore_regressors = ["ExtraTreesRegressor", "NuSVR", "QuantileRegressor", "SVR", "CCA", "GammaRegressor", "GaussianProcessRegressor", "IsotonicRegression",
                     "MultiOutputRegressor", "MultiTaskElasticNet", "MultiTaskElasticNetCV", "MultiTaskLasso", "MultiTaskLassoCV", "PLSCanonical", 
                     "RadiusNeighborsRegressor", "RegressorChain", "StackingRegressor", "KernelRidge", "ARDRegression", "TweedieRegressor"]
REGRESSORS = [
    est
    for est in all_estimators()
    if (issubclass(est[1], RegressorMixin) and (not (est[0] in ignore_regressors)))
]

for index, name in enumerate(cols_cible_type):
    df = load_df(name)
    # X = df.drop(["PumpSecondsOnSite_min", "PumpSecondsOnSite_mean", "PumpSecondsOnSite_max", "TurnoutTimeSeconds_min", "TurnoutTimeSeconds_mean",
    #               "TurnoutTimeSeconds_max", "TravelTimeSeconds_min", "TravelTimeSeconds_mean", "TravelTimeSeconds_max", "NumPumpsAttending"], axis=1)
    # display(X.head(2))
    # print(X.shape)

    # pca = PCA()
    # data_2D = pca.fit_transform(X)
    # display(data_2D)
    # print("La part de variance expliquée est", round(pca.explained_variance_ratio_.sum(), 5))

    # plt.figure()
    # plt.xlim(0,40)
    # plt.xlabel('Nombre de composantes')
    # plt.ylabel('Part de variance expliquée')
    # plt.axhline(y = 0.95, color ='r', linestyle = '--')
    # plt.plot(pca.explained_variance_ratio_.cumsum());
    # plt.show()

    # variance_expliquee = pca.explained_variance_ratio_
    # charges_factorielles = pca.components_
    # # Créez un DataFrame pour afficher les corrélations de chaque variable avec chaque axe
    # df_charges_factorielles = pd.DataFrame(
    #     charges_factorielles,
    #     columns=X.columns,
    #     #index=["Axe 1", "Axe 2", "Axe 3", "Axe 4", "Axe 5", "Axe 6"],
    # )

    # # Affichez le DataFrame des charges factorielles
    # display(df_charges_factorielles.head(2))

    # charges_factorielles = pca.components_
    # # Appelez la fonction pour tracer le cercle de corrélation
    # draw_correlation_circle(df_charges_factorielles, pca)

    # Auto ML
    df_limited = df[df.CalYear > 6]
    print(df_limited.shape)
    # Crée X sans les target
    cols_to_remove = [item for sublist in cols_cible for item in sublist] + ["NumPumpsAttending"]
    print(cols_to_remove)
    X = df_limited.drop(cols_to_remove, axis=1)
    display(X.head(2))
    print(X.shape)
    
    for col_cible in cols_cible[index]:
        print("cible", col_cible)
        y = df_limited[col_cible]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        myREGRESSORS = REGRESSORS
        print(myREGRESSORS)
        # myREGRESSORS = myREGRESSORS[4:6]
        reg = LazyRegressor(verbose=2, ignore_warnings=False, custom_metric=None, regressors=myREGRESSORS)
        models, predictions = reg.fit(X_train, X_test, y_train, y_test)
        display(models)
        models.to_csv(f"../data/_autoML_regressor_{col_cible}.csv", sep=";", index=True)
        # break
    # break

# tester automl avant et après PCA et aussi gridsearch pour comparer
# save du PCA

(1016665, 39)
['PumpSecondsOnSite_min', 'PumpSecondsOnSite_mean', 'PumpSecondsOnSite_max', 'TurnoutTimeSeconds_min', 'TurnoutTimeSeconds_mean', 'TurnoutTimeSeconds_max', 'TravelTimeSeconds_min', 'TravelTimeSeconds_mean', 'TravelTimeSeconds_max', 'NumPumpsAttending']


,CalYear,HourOfCall_0,HourOfCall_1,HourOfCall_2,HourOfCall_3,HourOfCall_4,PropertyType_0,PropertyType_1,PropertyType_2,PropertyType_3,PropertyType_4,PropertyType_5,Postcode_district_0,Postcode_district_1,Postcode_district_2,Postcode_district_3,Postcode_district_4,StopCode_0,StopCode_1,StopCode_2,StopCode_3,StopCode_4,Month_0,Month_1,Month_2,Month_3,DayOfWeek_0,DayOfWeek_1,DayOfWeek_2
574862,7,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1
574863,7,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1


(1016665, 29)
cible PumpSecondsOnSite_min
[('AdaBoostRegressor', <class 'sklearn.ensemble._weight_boosting.AdaBoostRegressor'>), ('BaggingRegressor', <class 'sklearn.ensemble._bagging.BaggingRegressor'>), ('BayesianRidge', <class 'sklearn.linear_model._bayes.BayesianRidge'>), ('DecisionTreeRegressor', <class 'sklearn.tree._classes.DecisionTreeRegressor'>), ('DummyRegressor', <class 'sklearn.dummy.DummyRegressor'>), ('ElasticNet', <class 'sklearn.linear_model._coordinate_descent.ElasticNet'>), ('ElasticNetCV', <class 'sklearn.linear_model._coordinate_descent.ElasticNetCV'>), ('ExtraTreeRegressor', <class 'sklearn.tree._classes.ExtraTreeRegressor'>), ('GradientBoostingRegressor', <class 'sklearn.ensemble._gb.GradientBoostingRegressor'>), ('HistGradientBoostingRegressor', <class 'sklearn.ensemble._hist_gradient_boosting.gradient_boosting.HistGradientBoostingRegressor'>), ('HuberRegressor', <class 'sklearn.linear_model._huber.HuberRegressor'>), ('KNeighborsRegressor', <class 'sklearn.neigh

  3%|▎         | 1/35 [00:47<26:43, 47.17s/it]

{'Model': 'AdaBoostRegressor', 'R-Squared': 0.11906588973138232, 'Adjusted R-Squared': 0.11894022956307293, 'RMSE': np.float64(1339.4178900741285), 'Time taken': 47.16416525840759}


...

{'Model': 'GradientBoostingRegressor', 'R-Squared': 0.17606630601772444, 'Adjusted R-Squared': 0.17594877662993635, 'RMSE': np.float64(1295.3600942703827), 'Time taken': 189.38852453231812}


 29%|██▊       | 10/35 [05:56<20:40, 49.60s/it]

{'Model': 'HistGradientBoostingRegressor', 'R-Squared': 0.19686673147999056, 'Adjusted R-Squared': 0.1967521691528873, 'RMSE': np.float64(1278.9047201029032), 'Time taken': 11.361153602600098}


 31%|███▏      | 11/35 [06:05<14:56, 37.37s/it]

{'Model': 'HuberRegressor', 'R-Squared': 0.03982290640609165, 'Adjusted R-Squared': 0.039685942683400754, 'RMSE': np.float64(1398.363413471799), 'Time taken': 9.626619338989258}


In [ ]:
import autosklearn.regression as autosk_reg
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

cols_cible_type = ["PumpSecondsOnSite", "TurnoutTimeSeconds", "TravelTimeSeconds"]
cols_cible = [
    ["PumpSecondsOnSite_min", "PumpSecondsOnSite_mean", "PumpSecondsOnSite_max"],
    ["TurnoutTimeSeconds_min", "TurnoutTimeSeconds_mean", "TurnoutTimeSeconds_max"],
    ["TravelTimeSeconds_min", "TravelTimeSeconds_mean", "TravelTimeSeconds_max"]
    # NumPumpsAttending??
]

def load_df(col_cible_type):
    df = pd.read_csv(f"../data/_df_ready_{col_cible_type}.csv", sep=";", low_memory=False)
    # Contrôle
    # display(df.head(3))
    # display(df.info())
    return df

for index, name in enumerate(cols_cible_type):
    df = load_df(name)
    X = df.drop(["PumpSecondsOnSite_min", "PumpSecondsOnSite_mean", "PumpSecondsOnSite_max", "TurnoutTimeSeconds_min", "TurnoutTimeSeconds_mean",
                  "TurnoutTimeSeconds_max", "TravelTimeSeconds_min", "TravelTimeSeconds_mean", "TravelTimeSeconds_max", "NumPumpsAttending"], axis=1)
    display(X.head(2))
    print(X.shape)
    
    # Auto ML
    df_limited = df[df.CalYear >= 10]
    X = df_limited.drop(["PumpSecondsOnSite_min", "PumpSecondsOnSite_mean", "PumpSecondsOnSite_max", "TurnoutTimeSeconds_min", "TurnoutTimeSeconds_mean",
                  "TurnoutTimeSeconds_max", "TravelTimeSeconds_min", "TravelTimeSeconds_mean", "TravelTimeSeconds_max", "NumPumpsAttending"], axis=1)
    for col_cible in cols_cible[index]:
        print("//////////////////////////////////////////////////////////////////////////////////////////")
        print(col_cible)
        y = df_limited[col_cible]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

        # Créer et entraîner Auto-sklearn pour la régression
        automl = autosk_reg.AutoSklearnRegressor(
            time_left_for_this_task=80000,  # Temps total alloué pour l'entraînement (en secondes)
            per_run_time_limit=6000,  # Temps maximum pour chaque modèle testé
            n_jobs=-1,  # Utilise tous les cœurs disponibles
            ensemble_kwargs = {'ensemble_size': 10},  # Nombre maximal de modèles dans l'ensemble final
            seed=42,
        )

        # Entraînement
        automl.fit(X_train, y_train)
        # Prédiction
        y_pred = automl.predict(X_test)
        # Évaluation des performan ces
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        print(f"Mean Squared Error: {mse}")
        print(f"R2 Score: {r2}")
        # Afficher un résumé détaillé des modèles et hyperparamètres
        print("\n********** Rapport détaillé des modèles **********\n")
        print(automl.show_models())
        print("\n********** Statistiques d'entraînement **********\n")
        print(automl.sprint_statistics())